In [ ]:
import pandas as pd
import numpy as np

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import datetime
from datetime import datetime

%matplotlib inline

import quandl
#quandl.ApiConfig.api_key = "*****"

In [ ]:

#mydata = quandl.get("FRED/GDP", start_date="1947-01-01", end_date="1974-01-01")
mydata = quandl.get("NSE/SPYL", start_date="2011-01-12", end_date="2013-01-01")


print(mydata)


Otro data sets para series se puede obtener del sitio https://faculty.chicagobooth.edu/ruey-s-tsay/teaching

Estas series fueron utilizadas en las diferentes ediciones del libro Analysis of Financial Time Series, de Ruey S. Tsay.

- Monthly IBM stock, VW index, EW index and S&P index returns: m-ibm3dx2608.csv

- Growth rate of U.S. quarterly real gnp: dgnp82.txt

- Daily values of S&P 500 index: d-sp55008.csv

- Monthly returns of VW index: VW_index_rets.csv

- Weekly 1-yr & 3-yr interest rates: w-gs1yr.csv & w-gs3yr.csv

- Monthly simple returns of Intel stock: Intel_returns.csv

- 10-minute FX log returns (Mark-Dollar): FX_logret_Mark_Dollar.csv

- Monthly excess returns of the S&P 500 index: S_P500_index.csv

In [ ]:
rates_1yr = pd.read_csv("Data/w-gs1yr.csv")
rates_3yr = pd.read_csv("Data/w-gs3yr.csv")

IBM_VW_EW_SyP_index = pd.read_csv("Data/m-ibm3dx2608.csv")
IBM_VW_EW_SyP_index['date'] = pd.to_datetime(IBM_VW_EW_SyP_index['date'].astype(str), format ='%Y%m%d' )

GNP = pd.read_csv("Data/dgnp82.csv")
S_P_index_d = pd.read_csv("Data/d-sp55008.csv")
S_P500_index = pd.read_csv("Data/S_P500_index.csv")

VW_Index_rets = pd.read_csv("Data/VW_Index_rets.csv")

FX_logret_mark_dollar = pd.read_csv("Data/FX_logret_Mark_Dollar.csv")

Intel_returns = pd.read_csv("Data/Intel_returns.csv")
Intel_returns['date'] = pd.to_datetime(Intel_returns['date'].astype(str), format='%Y%m%d')

In [ ]:
## La siguiente función puede utilizarse para analizar tres gráficos: la serie, la función 
## autocorrelación y la función de autocorrelación parcial

def tsplot(y, lags=None, figsize=(15, 10), style='bmh', titulo = 'Time Series Analysis Plots'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        #mpl.rcParams['font.family'] = 'Ubuntu Mono'
        layout = (3, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        qq_ax = plt.subplot2grid(layout, (2, 0))
        pp_ax = plt.subplot2grid(layout, (2, 1))
        
        y.plot(ax=ts_ax)
        ts_ax.set_title(titulo)
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)
        sm.qqplot(y, line='s', ax=qq_ax)
        qq_ax.set_title('QQ Plot')        
        scs.probplot(y, sparams=(y.mean(), y.std()), plot=pp_ax)

        plt.tight_layout()
    return 

In [ ]:
##Ejemplo

precios = mydata["Open"]

## log-retornos
lrets = np.log(precios/precios.shift(1)).dropna()
## retornos
rets = ((precios-precios.shift(1))/precios.shift(1)).dropna()


fig, [ax1, ax2]= plt.subplots(nrows = 2, ncols = 1, figsize=(15, 10))
ax1.plot(precios, color = 'r',label = 'Serie original')
ax1.legend(loc = 'best')
ax2.plot(lrets, label = 'Serie diferenciada')
ax2.legend(loc = 'best')
#plt.subplot(3,1,3)
#rets.plot()


plt.show()

In [ ]:
## Ejemplo
plt.figure()
smt.graphics.plot_acf(precios,lags=20, title = 'ACF precios')
smt.graphics.plot_acf(lrets, lags=20, title = 'ACF retornos')
plt.show()

In [ ]:
##Ejemplo: Clusters de volatilidad en tasas de interés a 1 año y 3 años

tasas1y = rates_1yr["rate"][1000:]
tasas3y = rates_3yr["rate"][1000:]
## log-retornos
lrets1y = np.log(tasas1y/tasas1y.shift(1)).dropna()
lrets3y = np.log(tasas3y/tasas3y.shift(1)).dropna()


fig, [ax1, ax2]= plt.subplots(nrows = 2, ncols = 1, figsize=(15, 10))
ax1.plot(lrets1y, color = 'r',label = 'Serie rates 1 year')
ax1.legend(loc = 'best')
ax2.plot(lrets3y, label = 'Serie rates 3 year')
ax2.legend(loc = 'best')
#plt.subplot(3,1,3)
#rets.plot()


plt.show()

In [ ]:
import statsmodels
statsmodels.tsa.stattools.adfuller(rets)

In [ ]:
tsplot(rets, lags = 30, titulo = "Retornos NSE/SPYL")

### Ruido blanco
Es un proceso $a_t$ con las siguientes propiedades:

-- Media $0$: $E[a_t] = 0$.

-- Varianza finita: $Var(a_t) = \sigma_\epsilon^2<\infty$. 

-- Autocovarianzas nulas: $Cov(a_t, a_{t-k})=0$.



### Actividad 

Construir un ruido blanco con varianza:

-- $\sigma_a = 1$

-- $\sigma_a = 0.5$

-- $\sigma_a = 0.01$

In [ ]:
T = 1000

B = pd.Series(index=range(T), dtype = 'object')
B.name = 'B'

for t in range(T):
    # Modificar los parámetros de la normal para modificar media y varianza.
    # 
    B[t] = np.random.normal(3,1/3)

plt.figure(figsize=(15,5))
plt.plot(B)

plt.xlabel('Tiempo')
plt.ylabel('Valores')
plt.legend(['Ruido blanco'])
smt.graphics.plot_acf(B, lags = 20)

plt.show()

## Para las series anteriores analizar la relación entre la media y el comportamiento de la serie.

In [ ]:
m = np.mean(B)
plt.figure(figsize=(15,7))
plt.plot(B)
plt.hlines(m, 0, l}en(B), linestyles='dashed', colors='r')
plt.xlabel('Tiempo')
plt.ylabel('Valores')
plt.legend(['Serie B', 'Media'])
plt.show()

# Caminata Aleatoria

In [ ]:
#np.random.seed(1)
T = 100

x = w = np.random.normal(0,1,size=T)
x[0]= 0
for t in range(1,T):
    x[t] = x[t-1] + w[t]

tsplot(x, lags = 20, titulo = "Caminata aleatoria")

# Tendencia lineal

In [ ]:
w = np.random.randn(250)
x = np.empty_like(w)

a0 = 100.
a1 = 0.5
for t in range(len(w)):
    x[t] = a0 + a1*t + w[t]
    
_ = tsplot(x, lags=30, titulo = "Tendencia lineal")  